In [26]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [27]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

import numpy as np # linear algebra
import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import pandas
import matplotlib.pyplot as plt
%matplotlib inline
! pip install plotly_express
import plotly_express as px

# Any results you write to the current directory are saved as output.

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score

import io
from datetime import datetime
import time

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn.metrics import accuracy_score , roc_auc_score , f1_score, precision_score , average_precision_score , recall_score


import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier , OneVsOneClassifier 
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_val_predict , LeaveOneOut , ShuffleSplit,RandomizedSearchCV

In [0]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [0]:
import os
os.chdir('/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureEngineering/')

In [30]:
!ls

bond_ratings.csv  finalDf_pca.csv	fund_specs.csv	     return_3year.csv
data.csv	  fund_allocations.csv	other_specs.csv      return_5year.csv
data_joined.csv   fund_config.csv	primary_keys_df.csv
data_tst_ids.csv  fund_ratios.csv	return_10year.csv


In [0]:
## copying the files to feature selection folder
!cp data.csv  '/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/'
!cp data_joined.csv  '/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/'

In [0]:
os.chdir('/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/')

In [33]:
!ls

best_columns.csv
data.csv
data_joined.csv
data_tst_ids.csv
df_ns_df_sorted_annova.csv
df_ns_df_sorted_chi2.csv
df_ns_df_sorted_FI.csv
Feature-Selection-Methods.png
finalDf_pca.csv
Hackathon_GreatLearning_Great_Stone_Rating_DeepNeural.h5
sample_submission.csv


In [34]:
data = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/data.csv")
data_joined = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/data_joined.csv")
best_columns = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/best_columns.csv")
finalDf_pca = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/finalDf_pca.csv")
sample_submission = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/sample_submission.csv")
data_tst_ids = pd.read_csv("/gdrive/My Drive/Hackathon/GREATLEARNING-APRIL/FeatureSelection/data_tst_ids.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (7,9,10,18,57,80) have mixed types.Specify dtype option on import or set low_memory=False.



In [35]:
col_int = data.drop(columns=['greatstone_rating'],axis=1).select_dtypes(exclude=('category','object')).columns
col_id = ['tag','id','fund_id']
col_tgt = ['greatstone_rating']
col_cat = data.drop(columns=['greatstone_rating'],axis=1).select_dtypes(include=('category','object')).columns
print(len(col_int)+len(col_id)+len(col_tgt)+len(col_cat))
print('col_int :- ',len(col_int),'| col_cat :-',len(col_cat),'| col_id :-',len(col_id),'| col_tgt :-',len(col_tgt))
print(col_cat)

958
col_int :-  954 | col_cat :- 0 | col_id :- 3 | col_tgt :- 1
Index([], dtype='object')


In [36]:
data.isnull().sum().sum()

5000

In [37]:
data.shape

(45716, 955)

.values will give the values in an array. (shape: (n,1)

.ravel will convert that array shape to (n, )


In [38]:
set1 = set(best_columns.FI)
set2 = set(best_columns.CHI2)
set3 = set(best_columns.ANNOVA)
bst_col = list(set1|set2|set3)
print(len(bst_col))

93


In [39]:
################################ OUTLIERS DETECTION  #################################
out_compare = pd.DataFrame(columns=['Column_name'])

col=bst_col

for i in range(len(col)):
  factor=2.5
  upper_lmt=data[col[i]].mean()+data[col[i]].std()*factor
  lower_lmt=data[col[i]].mean()-data[col[i]].std()*factor
  out_compare.loc[i,'Column_name']=col[i]
  out_compare.loc[i,'upper_lmt']=upper_lmt
  out_compare.loc[i,'lower_lmt']=lower_lmt
  out_compare.loc[i,'upper_outlier(%)']='{0:=5.2f} %'.format((data[(data[col[i]]>upper_lmt)==True][col[i]].count()/data[col[i]].count())*100)
  out_compare.loc[i,'lower_outlier(%)']='{0:<5.2f} %'.format((data[(data[col[i]]<lower_lmt)==True][col[i]].count()/data[col[i]].count())*100)


out_compare = out_compare.sort_values(by=['upper_outlier(%)','lower_outlier(%)'],ascending=False)
out_compare.set_index('Column_name',inplace=True)
out_compare

#Capping the outlier rows with Percentiles
#upper_lim = data['column'].quantile(.95)
#lower_lim = data['column'].quantile(.05)

,upper_lmt,lower_lmt,upper_outlier(%),lower_outlier(%)
Column_name,,,,
is_month_end,0.893052,-0.649392,9.40 %,0.00 %
aa_rating,29.529866,-21.388486,4.35 %,0.00 %
category_43,0.505012,-0.426176,3.94 %,0.00 %
2011_return_category,13.298608,-5.765797,3.93 %,0.00 %
cash_percent_of_portfolio,43.412827,-26.534914,3.48 %,0.00 %
...,...,...,...,...
Weekday_Name,7.758233,-2.278549,0.00 %,0.00 %
category,129.203636,-20.599442,0.00 %,0.00 %
is_leap_year,1.305046,-0.772488,0.00 %,0.00 %


In [40]:
#################### OUTLIERS TREATMENT and Verifying it after the treatment     ##############################

col=bst_col

out_fx_compare = pd.DataFrame(columns=['Column_name'])

for i in range(len(col)):
  
  #repl_value = out_compare.loc[col[i],'upper_lmt']
  repl_value = data[col[i]].mean()
  #print('column {}:- has mean value {}'.format(col[i],repl_value))
  data[col[i]].replace(data[(data[col[i]]>out_compare.loc[col[i],'upper_lmt'])==True][col[i]],repl_value,inplace=True)  ## value replacement upper limit
  data[col[i]].replace(data[(data[col[i]]<out_compare.loc[col[i],'lower_lmt'])==True][col[i]],repl_value,inplace=True) ## value replacement lower limit
  
  out_fx_compare.loc[i,'Column_name']=col[i]
  out_fx_compare.loc[i,'upper_lmt_old']=out_compare.loc[col[i],'upper_lmt']
  out_fx_compare.loc[i,'lower_lmt_old']=out_compare.loc[col[i],'lower_lmt']
  out_fx_compare.loc[i,'upper_outlier_nw(%)']='{0:=5.2f} %'.format((data[(data[col[i]]>out_compare.loc[col[i],'upper_lmt'])==True][col[i]].count()/data[col[i]].count())*100)
  out_fx_compare.loc[i,'lower_outlier_nw(%)']='{0:<5.2f} %'.format((data[(data[col[i]]<out_compare.loc[col[i],'lower_lmt'])==True][col[i]].count()/data[col[i]].count())*100)
 
out_fx_compare = out_fx_compare.sort_values(by=['upper_lmt_old'],ascending=False)
out_fx_compare.set_index('Column_name',inplace=True)
out_fx_compare


,upper_lmt_old,lower_lmt_old,upper_outlier_nw(%),lower_outlier_nw(%)
Column_name,,,,
total_assets,4.430565e+10,-3.806358e+10,0.00 %,0.00 %
mmc,1.053445e+05,-5.160337e+04,0.00 %,0.00 %
pc_ratio,1.067495e+03,-1.034526e+03,0.00 %,0.00 %
parent_company,8.848430e+02,-1.101117e+02,0.00 %,0.00 %
10yrs_treynor_ratio_fund,3.116803e+02,-2.788515e+02,0.00 %,0.00 %
...,...,...,...,...
parent_company_537,7.507865e-02,-7.297960e-02,0.00 %,0.00 %
parent_company_680,7.090372e-02,-6.888975e-02,0.00 %,0.00 %
parent_company_233,6.783914e-02,-6.594290e-02,0.00 %,0.00 %


In [0]:
###############################################################  KFOLD VALIDATION STRATIFIED ######################################################################

from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
  

data_train = data[~data.greatstone_rating.isnull()]
data_tst = data[data.greatstone_rating.isnull()]
data_train.shape

models = []
#models.append(('LogisticRegression', LogisticRegression()))
#models.append(('GaussianNB', naive_bayes.GaussianNB()))
#models.append(('KNeighborsClassifier', KNeighborsClassifier()))
#models.append(('SVC', SVC()))
#models.append(('DecisionTreeClassifier', DecisionTreeClassifier(max_depth=20)))
models.append(('RandomForestClassifier', RandomForestClassifier(n_estimators=450,bootstrap=False,max_depth=None,max_features='auto',min_samples_leaf=1,min_samples_split=2)))
models.append(('AdaBoostClassifier', ensemble.AdaBoostClassifier()))
models.append(('BaggingClassifier', ensemble.BaggingClassifier()))
models.append(('GradientBoostingClassifier', ensemble.GradientBoostingClassifier()))
#models.append(('naive_bayes.BernoulliNB',  naive_bayes.BernoulliNB()))
#models.append(('MLPClassifier',  MLPClassifier(hidden_layer_sizes = [100]*5)))
models.append(('XGBClassifier', XGBClassifier()))

matrix_tbl = pd.DataFrame()

i=0
k=0
n_splits = 8
best_col = ['FI','ANNOVA','CHI2']

for k in range(len(best_col)):

  import pytz
  from datetime import datetime
  # assuming now contains a timezone aware datetime
  now = datetime.now()
  tz = pytz.timezone('Asia/Kolkata')
  your_now = now.astimezone(tz)
  print (your_now)

  print("Feature Selection Technique used :-",best_col[k], "\n")

  X_0 = data_train[best_columns[best_col[k]]]      
  y_0 = data_train[col_tgt]





  np.random.seed(7)
  kfold = StratifiedKFold(n_splits=n_splits,random_state=None)
  #kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=21312)
  # KFold,GroupKFold  , ShuffleSplit, GroupShuffleSplit  ,StratifiedKFold, StratifiedShuffleSplit
                                     
                                     
  


  ################################################################################################################################################################


    
  for name, model in models:
      cvscore_train = []
      cvscore_test = []
      for train, test in kfold.split(X_0, y_0):
            X_train = X_0.iloc[train,:]
            X_test = X_0.iloc[test,:]
            X_train_minmax_0 = mm_scaler.fit_transform(X_train)
            X_valid_minmax_0 = mm_scaler.fit_transform(X_test)
            #print(X_train_minmax_0.shape[1])
            y_train_0 = y_0.iloc[train,:]
            y_valid_0 = y_0.iloc[test,:]
            y_train_0 = y_train_0.values.ravel()
            y_valid_0 = y_valid_0.values.ravel()

            # X_train_0,X_valid_0,y_train_0,y_valid_0 = train_test_split(X_0,y_0,test_size=0.3,random_state=576)
            #print("TRAIN:", train, "TEST:", test)
            print(X_train_minmax_0.shape,y_train_0.shape,X_valid_minmax_0.shape,y_valid_0.shape)

            print("\n")
            #model = model.fit(X_train_minmax_0, y_train_0)
            model_OVRC = OneVsRestClassifier(model) # OneVsOneClassifier ,OneVsRestClassifier
            model_OVRC.fit(X_train_minmax_0, y_train_0)
            print("for the model {} train score is {} and test score is {}".format(name,model_OVRC.score(X_train_minmax_0, y_train_0),model_OVRC.score(X_valid_minmax_0, y_valid_0)))
            #print("Train score :- ", model_OVRC.score(X_train_minmax_0, y_train_0))
            #print("Test score :- ", model_OVRC.score(X_valid_minmax_0, y_valid_0))
            cvscore_train.append(model_OVRC.score(X_train_minmax_0, y_train_0))
            cvscore_test.append(model_OVRC.score(X_valid_minmax_0, y_valid_0))

      print("Mean Accuracy (+/- Std Dev):-  %.2f%% (+/- %.2f%%)" % (np.mean(cvscore_test), np.std(cvscore_test)))
      print("\n")

      matrix_tbl.loc[i,'Feature'] = best_col[k]
      matrix_tbl.loc[i,'Number_of_columns'] = X_train_minmax_0.shape[1]
      matrix_tbl.loc[i,'name'] = name
      matrix_tbl.loc[i,'Mean_Train_score'] = np.mean(cvscore_train)
      matrix_tbl.loc[i,'Train_score_Std_dev'] = np.std(cvscore_train)
      matrix_tbl.loc[i,'Mean_Test_score'] = np.mean(cvscore_train)
      matrix_tbl.loc[i,'Test_score_Std_dev'] = np.std(cvscore_train)
      i=i+1
  
print("\n")

import pytz
from datetime import datetime
# assuming now contains a timezone aware datetime
now = datetime.now()
tz = pytz.timezone('Asia/Kolkata')
your_now = now.astimezone(tz)
print (your_now)

print("**************************************************\n")



2020-05-11 08:50:23.434417+05:30
Feature Selection Technique used :- FI 

(35626, 50) (35626,) (5090, 50) (5090,)




In [0]:
matrix_tbl_pivot =  pd.pivot_table(matrix_tbl,index=['Feature','name'])

In [0]:
matrix_tbl_pivot

# **MODEL TRAINING ON BEST MODEL**

In [0]:
###############################################################  KFOLD VALIDATION STRATIFIED ######################################################################

from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
  
attributes= best_columns.FI
n_splits = 8

data_train = data[~data.greatstone_rating.isnull()]
data_tst = data[data.greatstone_rating.isnull()]
data_train.shape


import pytz
from datetime import datetime
# assuming now contains a timezone aware datetime
now = datetime.now()
tz = pytz.timezone('Asia/Kolkata')
your_now = now.astimezone(tz)
print (your_now)


X_0 = data_train[attributes]      
y_0 = data_train[col_tgt]

cvscore_train = []
cvscore_test = []

np.random.seed(7)
kfold = StratifiedKFold(n_splits=n_splits,random_state=None)
#kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=21312)
  


  ################################################################################################################################################################
model_OVRC = OneVsRestClassifier(RandomForestClassifier(n_estimators=450,bootstrap=False,max_depth=30,max_features='auto',min_samples_leaf=1,min_samples_split=2)) # OneVsOneClassifier ,OneVsRestClassifier
for train, test in kfold.split(X_0, y_0):
        X_train = X_0.iloc[train,:]
        X_test = X_0.iloc[test,:]
        X_train_minmax_0 = mm_scaler.fit_transform(X_train)
        X_valid_minmax_0 = mm_scaler.fit_transform(X_test)
        #print(X_train_minmax_0.shape[1])
        y_train_0 = y_0.iloc[train,:]
        y_valid_0 = y_0.iloc[test,:]
        y_train_0 = y_train_0.values.ravel()
        y_valid_0 = y_valid_0.values.ravel()

        # X_train_0,X_valid_0,y_train_0,y_valid_0 = train_test_split(X_0,y_0,test_size=0.3,random_state=576)
        #print("TRAIN:", train, "TEST:", test)
        print(X_train_minmax_0.shape,y_train_0.shape,X_valid_minmax_0.shape,y_valid_0.shape)

        print("\n")
        model_OVRC.fit(X_train_minmax_0, y_train_0)
        print("for the model  train score is {} and test score is {}".format(model_OVRC.score(X_train_minmax_0, y_train_0),model_OVRC.score(X_valid_minmax_0, y_valid_0)))
        cvscore_train.append(model_OVRC.score(X_train_minmax_0, y_train_0))
        cvscore_test.append(model_OVRC.score(X_valid_minmax_0, y_valid_0))

print("Mean Accuracy (+/- Std Dev):-  %.2f%% (+/- %.2f%%)" % (np.mean(cvscore_test), np.std(cvscore_test)))


import pytz
from datetime import datetime
# assuming now contains a timezone aware datetime
now = datetime.now()
tz = pytz.timezone('Asia/Kolkata')
your_now = now.astimezone(tz)
print (your_now)



# **MODEL PREDICTIONS ON TEST DATA**

In [0]:
####################################################***** PREDICTIONS ********############################################################################

In [0]:
data_tst = data[data.greatstone_rating.isnull()]

data_tst_df = data_tst[attributes]

data_tst_minmax_0 = mm_scaler.fit_transform(data_tst_df)

model_OVRC_predict = model_OVRC.predict(data_tst_minmax_0)


prediction = pd.DataFrame(model_OVRC_predict,columns=['greatstone_rating'])

print("prediction.isnull() :- ", prediction.isnull().sum())


prediction_df = prediction.reset_index().groupby('greatstone_rating').count().reset_index().rename(columns={'index':'count'}).sort_values(by='greatstone_rating',ascending=True)


df1 =pd.DataFrame()

df1['tag'] = data_tst_ids.tag
df1['id'] = data_tst_ids.id
df1['fund_id'] = data_tst_ids.fund_id
df1['greatstone_rating'] = prediction.greatstone_rating



df2 = pd.merge(sample_submission.drop(columns='greatstone_rating',axis=1),df1,how='left',on='fund_id')


df2.drop(columns=['tag','id'],axis=1,inplace=True)


df2.reset_index().groupby('greatstone_rating').count().reset_index().rename(columns={'index':'count'}).sort_values(by='greatstone_rating',ascending=True)


from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


print(prediction_df)


In [0]:
create_download_link(df2)